In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import pandas
import numpy as np
import tensorflow as tf
import random
import os
import pickle

In [ ]:
import rtdl
sys.modules['rtdl.rtdl'] = rtdl

In [ ]:
data_to_save = pickle.load(open("real_background.pkl", "rb"))

background_x = data_to_save['background_x']
background_y_lbl = data_to_save['background_y_lbl']
pad_in = data_to_save['pad_in']
wrapper_df = data_to_save['wrapper_df']
background_sizes = data_to_save['background_sizes']
col_names = data_to_save['col_names']
to_explain_shape = data_to_save['to_explain_shape']
test_comb = data_to_save['test_comb']

In [ ]:
#Using GPU

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["TF_CUDNN_DETERMINISTIC"]="1"
physical_devices = tf.config.list_physical_devices('GPU')
for gpu_dev in physical_devices:
    tf.config.experimental.set_memory_growth(gpu_dev, True)
physical_devices

In [ ]:
from explain_icu import TheWrapper

import sys
sys.path.append(".")
sys.path.append("../required_libs")
sys.path.append("../required_libs/TSInterpret")
sys.path.append("../required_libs/TSInterpret/InterpretabilityModels")
sys.path.append("../required_libs/TSInterpret/InterpretabilityModels/Saliency")

from ex_methods import gen_explanations
import pathlib

import pandas

from ex_methods import do_Anchors, do_WindowSHAP, do_COMTE

from required_libs.anchor import anchor_tabular

from required_libs.WindowSHAP.windowshap import StationaryWindowSHAP

from required_libs.TSInterpret.InterpretabilityModels.counterfactual.COMTECF import COMTECF

In [ ]:
background_sizes = [x.shape for x in background_x]
model = TheWrapper(wrapper_df, background_sizes, col_names, to_explain_shape, n_workers=8, multiproc=True)

In [ ]:
pred_vars = wrapper_df['pred_vars']
pred_var_locs = [i for i in range(len(col_names)) if col_names[i] in pred_vars]

In [ ]:
def get_implemented_methods():
    #List of the methods currently implemented for use
    impl_dict = {
        "Anchors": {"function": do_Anchors, "result_store": {"explanations": [], "time_taken": [], "random_seed": [], "item_index": [], "samples_explained": []}},
    }
    return impl_dict

In [ ]:
#GENERATE THE EXPLANATIONS

available_exp_methods = get_implemented_methods()

#Select which of the methods to use based on the configuration
methods_to_use = {m:v for m, v in available_exp_methods.items()}

In [ ]:
temp = pandas.read_pickle("new_input2.pkl")

In [ ]:
id_to_int_map = {k: int(k+10000000) for k in temp['wrapper_df']['combined_dat']['encounter_id'].unique()}

In [ ]:
def find_records(all_data, list_encids):
    return all_data[all_data['encounter_id'].isin(list_encids)]

In [ ]:
BASE_FOLDER_NAME = 'trial44_vis_outs'
OUTPUT_FOLDERS = ['outs0']#
OUTPUT_FOLDERS = [BASE_FOLDER_NAME + "/" + x for x in OUTPUT_FOLDERS]

In [ ]:
for r_seed, outfolder in enumerate(OUTPUT_FOLDERS):
    print(f"------------OUTFOLDER {outfolder} -------------------")
    r_seed += 123
    tf.random.set_seed(r_seed)
    random.seed(r_seed)
    np.random.seed(r_seed)
    
    collected_test_recs = []
    for test_encid in id_to_int_map.keys():
        rec = find_records(temp['wrapper_df']['combined_dat'], [test_encid])
        rec['encounter_id'] = id_to_int_map[test_encid]

        rec = np.expand_dims(rec.to_numpy(), axis=0)
        x_in = rec
        y_in = [1] #Using positive cases only

        x_in_length = x_in.shape[1]
        background_x_truncated = background_x[:, -x_in_length:, :]
        NUM_USED=500 #actual num_used * 2
        background_x_truncated = np.concatenate([background_x_truncated[:NUM_USED], background_x_truncated[-NUM_USED:]], axis=0)

        model.to_explain_shape = x_in.shape[1:]
        model.col_names = col_names
        
        explanation_config = {
        'output_folder': outfolder,
        'model': model,
        'feature_names': pred_vars,
        'model_type': 'lstm',
        'what_is_second_dim': 'time',
        'all_names':col_names,
         'pred_var_locs': pred_var_locs,
        'background_data': {
            'x': background_x_truncated,
            'y': background_y_lbl,
            },
        'experiment_config':{
            'device': 'cpu',
            'categorical_features':[],
            },
        'comte':{
            'max_n_feats': 6,
            'tau_confidence': 0.95,
            'num_background_used': 200
            },
        'windowshap':{
            'window_len': 2,
            'num_background_used': 200
            },
        'anchors':{
            'tau': 0.25,
            'delta': 0.25,
            'num_background_used': 200,
            'batch_size': 256,
            'threshold': 0.75,
            }
        }

        gen_explanations(explanation_config, methods_to_use, x_in, y_in)